In [1]:
# adding imports for this project
import notebook
import numpy.fft
from scipy.io import wavfile
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import soundfile
import math
import time
from scipy.signal import spectrogram, lfilter, freqz, tf2zpk, find_peaks, butter, buttord, normalize

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# reading original signal
data, fs = soundfile.read('../audio/xokruc00_original.wav')
data.min(), data.max()
t = data.size / fs

print("Time : ", t, "s")
print("Signal length : ", data.size)

print("Min : ", data.min())
print("Max : ", data.max())

In [ ]:
timex = np.arange(data.size) / fs
plt.figure(figsize=(13,5))
plt.plot(timex,data)

plt.gca().set_xlabel('$t[s]$')
plt.gca().set_title('Sound signal')

plt.tight_layout()

In [ ]:
# 4.2

start = 0
end = 1024
frame = []
index = 0
while end < data.size:
    frame.append(data[start:end])

    start = start + 512
    end = end + 512
    index += 1

    if index == 40 :
        chosenStart = (start - 512)/fs
        chosenEnd = (end - 512)/fs

print("frames: ", len(frame))
print("start ", start)
print("end ", end)

print("start ", chosenStart)
print("end ", chosenEnd)

print(frame[40])

time = np.arange(frame[40].size)/fs+chosenStart
print("TIME ",time)
plt.figure(figsize=(13,3))
plt.plot(np.arange(frame[40].size)/fs+chosenStart,frame[40])
plt.gca().set_xlabel('$t[s]$')
plt.gca().set_title('Frame number 41')
plt.tight_layout()

In [ ]:
# 4.3
def myDFT(function):
    N = function.size
    result = []
    num = 0
    k,n = 0,0
    for k in range(1024) :
        for n in range(1024) :
            realPart = math.cos(((2 * math.pi) / N) * k * n)
            imagPart = math.sin(((2 * math.pi) / N) * k * n)
            complexNum = complex(realPart, -imagPart)
            num += function[n] * complexNum
        result.append(num)
    return result


fftresult = numpy.fft.fft(data)
myFrameFFT = numpy.fft.fft(frame[40])
mytestFFT = myDFT(frame[40])

print(mytestFFT[0])
print(myFrameFFT[0])

half = fftresult.size//2

times = np.arange(0.01, 0.98, 0.01)
freqs = np.arange(0, 1024/2) / 1024 * 16000

plt.figure(figsize=(17,3))
yaxis = myFrameFFT[0:(myFrameFFT.size//2)]
xaxis = np.arange(0, fs/2, fs/1024)

plt.plot(xaxis, np.abs(yaxis))
plt.gca().set_xlabel('$F[Hz]$')
plt.gca().set_title('Graph DFT')


In [ ]:


f,t,sgr = spectrogram(data, fs, nperseg=1024, noverlap=512)
spr_log = 10 * np.log10(abs(sgr)**2)



plt.figure(figsize=(13,5))
plt.pcolormesh(t, f, spr_log)
plt.gca().set_xlabel('Time [s]')
plt.gca().set_ylabel('Frequency [Hz]')
cbar = plt.colorbar()

plt.tight_layout()

In [ ]:
# function for searching peaks in signal
# arguments which I used are : signal input, height of signal (if signal is higher than this value it is considered as peak, I chose 100 because it is high enough to be considered as a peak)
# documentation for this function : https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html
peaks ,_ = find_peaks(numpy.abs(myFrameFFT[0:(myFrameFFT.size//2)]), height=2)


print(peaks)

f1 = peaks[0] * fs/1024
f2 = peaks[1] * fs/1024
f3 = peaks[2] * fs/1024
f4 = peaks[3] * fs/1024

print("Frequencies of our peaks are : ")
print("f1 :", f1, "Hz")
print("f2 :", f2, "Hz")
print("f3 :", f3, "Hz")
print("f4 :", f4, "Hz")

print("Now we need to check if they are harmonically related")
print("f2 :", f2, "== f1 * 2 :", f1*2)
print("f3 :", f3, "== f1 * 2 :", f1*3)
print("f4 :", f4, "== f1 * 2 :", f1*4)

print("Even through small offset I consider them as harmonically related")

plt.figure(figsize=(17,3))

plt.plot(numpy.arange(0, fs//2, fs/1024),numpy.abs(myFrameFFT[0:myFrameFFT.size//2]))

plt.plot(peaks*(16000//1024),numpy.abs(myFrameFFT[peaks]), "x")

plt.show()


In [ ]:
# 4.6 Generating signal

samples = []
for i in range(data.size):
    samples.append(i/fs)

outCos1 = np.cos(2 * np.pi * f1 * np.array(samples))
outCos2 = np.cos(2 * np.pi * f2 * np.array(samples))
outCos3 = np.cos(2 * np.pi * f3 * np.array(samples))
outCos4 = np.cos(2 * np.pi * f4 * np.array(samples))

outCosSum = outCos1 + outCos2 + outCos3 + outCos4

print("max: ", outCosSum.max())
print("min: ", outCosSum.min())

norm = outCosSum / outCosSum.max()

print("max: ", norm.max())
print("min: ", norm.min())

soundfile.write("../audio/4cos.wav", (norm * np.iinfo(np.int16).max).astype(np.int16), fs)

f,t,sgr = spectrogram(outCosSum, fs, nperseg=1024, noverlap=512)
spr_log = 10 * np.log10(abs(sgr)**2)

plt.figure(figsize=(13,5))
plt.pcolormesh(t, f, spr_log)
plt.gca().set_xlabel('Time [s]')
plt.gca().set_title('Spectrogram of 4cos.wav generated audio file')
plt.gca().set_ylabel('Frequence [Hz]')
cbar = plt.colorbar()


In [ ]:
#4.7
def createFilter(frequency):

    N, Wn = buttord([(frequency-90)/(fs/2), (frequency+90)/(fs/2)], [(frequency-30)/(fs/2), (frequency+30)/(fs/2)], 30, 50)
    print("N: ", N)
    print("Wn: ", Wn)
    b, a = butter(N, Wn, 'bandstop')

    return b, a

b1, a1 = createFilter(f1)
b2, a2 = createFilter(f2)
b3, a3 = createFilter(f3)
b4, a4 = createFilter(f4)

z1, p1, k1 = tf2zpk(b1, a1)
z2, p2, k2 = tf2zpk(b2, a2)
z3, p3, k3 = tf2zpk(b3, a3)
z4, p4, k4 = tf2zpk(b4, a4)

In [ ]:
plt.figure(figsize=(6,5.25))

# graf - časť jednotková kružnica
ang = np.linspace(0, 2*np.pi,100)
plt.plot(np.cos(ang), np.sin(ang))

plt.scatter(np.real(z1), np.imag(z1), marker='o', facecolors='none', edgecolors='r', label='zeros')
plt.scatter(np.real(p1), np.imag(p1), marker='x', color='g', label='poles')

plt.legend(loc='upper left')

plt.scatter(np.real(z2), np.imag(z2), marker='o', facecolors='none', edgecolors='r', label='zeros')
plt.scatter(np.real(p2), np.imag(p2), marker='x', color='g', label='poles')

plt.scatter(np.real(z3), np.imag(z3), marker='o', facecolors='none', edgecolors='r', label='zeros')
plt.scatter(np.real(p3), np.imag(p3), marker='x', color='g', label='poles')

plt.scatter(np.real(z4), np.imag(z4), marker='o', facecolors='none', edgecolors='r', label='zeros')
plt.scatter(np.real(p4), np.imag(p4), marker='x', color='g', label='poles')

plt.gca().set_xlabel('Real component $\mathbb{R}\{$z$\}$')
plt.gca().set_ylabel('Imaginary component  $\mathbb{I}\{$z$\}$')

plt.grid(alpha=0.5, linestyle='--')

plt.tight_layout()

In [ ]:
w1, H1 = freqz(b1, a1)
w2, H2 = freqz(b2, a2)
w3, H3 = freqz(b3, a3)
w4, H4 = freqz(b4, a4)

_, ax = plt.subplots(1, 2, figsize=(13,5))

ax[0].plot(w1 / 2 / np.pi * fs, np.abs(H1))
ax[0].set_xlabel('Frequency [Hz]')
ax[0].set_title('Frequency response module  f1 $|H(e^{j\omega})|$')

ax[1].plot(w1 / 2 / np.pi * fs, np.angle(H1))
ax[1].set_xlabel('Frequency [Hz]')
ax[1].set_title('Frequency response argument f1 $\mathrm{arg}\ H(e^{j\omega})$')

for ax1 in ax:
    ax1.grid(alpha=0.5, linestyle='--')

plt.tight_layout()

_, ax = plt.subplots(1, 2, figsize=(13,5))

ax[0].plot(w2 / 2 / np.pi * fs, np.abs(H2))
ax[0].set_xlabel('Frequency [Hz]')
ax[0].set_title('Frequency response module  f2 $|H(e^{j\omega})|$')

ax[1].plot(w2 / 2 / np.pi * fs, np.angle(H2))
ax[1].set_xlabel('Frequency [Hz]')
ax[1].set_title('Frequency response argument f2 $\mathrm{arg}\ H(e^{j\omega})$')

for ax1 in ax:
    ax1.grid(alpha=0.5, linestyle='--')

plt.tight_layout()

_, ax = plt.subplots(1, 2, figsize=(13,5))

ax[0].plot(w3 / 2 / np.pi * fs, np.abs(H3))
ax[0].set_xlabel('Frequency [Hz]')
ax[0].set_title('Frequency response module  f3 $|H(e^{j\omega})|$')

ax[1].plot(w3 / 2 / np.pi * fs, np.angle(H3))
ax[1].set_xlabel('Frequency [Hz]')
ax[1].set_title('Frequency response argument f3 $\mathrm{arg}\ H(e^{j\omega})$')

for ax1 in ax:
    ax1.grid(alpha=0.5, linestyle='--')

plt.tight_layout()

_, ax = plt.subplots(1, 2, figsize=(13,5))
ax[0].plot(w4 / 2 / np.pi * fs, np.abs(H4))
ax[0].set_xlabel('Frequency [Hz]')
ax[0].set_title('Frequency response module f4 $|H(e^{j\omega})|$')

ax[1].plot(w4 / 2 / np.pi * fs, np.angle(H4))
ax[1].set_xlabel('Frequency [Hz]')
ax[1].set_title('Frequency response argument f4 $\mathrm{arg}\ H(e^{j\omega})$')

for ax1 in ax:
    ax1.grid(alpha=0.5, linestyle='--')

plt.tight_layout()

In [ ]:
sf = lfilter(b1, a1, data)
sf = lfilter(b2, a2, sf)
sf = lfilter(b3, a3, sf)
sf = lfilter(b4, a4, sf)

soundfile.write("../audio/clean_bandstop.wav", (sf * np.iinfo(np.int16).max).astype(np.int16) , fs)

f,t,sgr = spectrogram(sf, fs, nperseg=1024, noverlap=512)
spr_log = 10 * np.log10(abs(sgr)**2)

plt.figure(figsize=(10,5))
plt.pcolormesh(t, f, spr_log)
plt.gca().set_xlabel('Time [s]')
plt.gca().set_ylabel('Frequency [Hz]')
cbar = plt.colorbar()

plt.tight_layout()
